In [9]:
import requests
import zipfile
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta
import pandas as pd
from ggplot import *
import sqlite3

C:\Users\David\Anaconda2\lib\site-packages\ggplot\utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
C:\Users\David\Anaconda2\lib\site-packages\ggplot\stats\smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
C:\Users\David\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [24]:
q_base = 'https://developer.nrel.gov/api/pvwatts/v5.xml?'
# Los Angeles: q_main = 'lat=34.02&lon=-118.45&system_capacity=4&azimuth=180&tilt=40&array_type=1&module_type=1&losses=10&timeframe=hourly&'
q_main = 'lat=36.783&lon=-119.717&system_capacity=4&azimuth=180&tilt=40&array_type=1&module_type=1&losses=10&timeframe=hourly&'
api_key = 'api_key='

In [25]:
url = q_base + q_main + api_key

In [22]:
### Enter API and abbreviated place name (place_tag)
def pvwatts_query(url):
    r = requests.get(url)
    open('extract','wb').write(r.content)
    tree = ET.fromstring(r.content)
    # Extract ac output for each hour in the year
    gen = []
    for data in tree.iter('ac'):
        gen.append(data.text)
        
    ### Gen includes parent as first entry (which is empty here; remove; length should be 0)
    del(gen[0])
    len(gen)
    
    ### Convert entries into dataframe with dates
    start = datetime.strptime('01/01/2017 00','%m/%d/%Y %H')
    end = datetime.strptime('12/31/2017 23','%m/%d/%Y %H')
    
    delta = end - start
    
    ### Generate datetime variables for each out between the start and end dates
    hour_steps = []

    for i in range((delta.days + 1)*24):
        hour_steps.append(start + timedelta(hours = i))
    
    df_gen = pd.DataFrame(
    {'date_time':hour_steps,
    'generation': gen})
    
    ### Create hour of the day variable
    df_gen['hour'] = df_gen['date_time'].apply(lambda x: datetime.strftime(x,'%H'))
    
    df_gen['generation'] = df_gen['generation'].astype('float')
    
    return df_gen

In [26]:
fresno_gen = pvwatts_query(url)

In [27]:
fresno_gen.head(n=15)

,date_time,generation,hour
0,2017-01-01 00:00:00,0.000,00
1,2017-01-01 01:00:00,0.000,01
2,2017-01-01 02:00:00,0.000,02
3,2017-01-01 03:00:00,0.000,03
4,2017-01-01 04:00:00,0.000,04
5,2017-01-01 05:00:00,0.000,05
6,2017-01-01 06:00:00,0.000,06
7,2017-01-01 07:00:00,31.052,07
8,2017-01-01 08:00:00,261.520,08
9,2017-01-01 09:00:00,915.334,09


In [28]:
conn = sqlite3.connect('pv_production_CA.db')

In [29]:
cur = conn.cursor()

In [30]:
fresno_gen.to_sql('fres_gen',conn)

In [31]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[(u'la_gen',), (u'redd_gen',), (u'fres_gen',)]


In [32]:
conn.close()